# 수집된 뉴스 기사 및 댓글에 대한 감정 분석
## * Word2Vec
* 데이터 
> 2017년 12월 1일부터 2018년 2월 1일까지 63일간 [네이버](http://www.naver.com)와 [다음](http://www.daum.net)의 랭킹뉴스와 뉴스의 댓글을 크롤링함.

In [1]:
import pickle
import html
import multiprocessing
from collections import namedtuple, OrderedDict
import re
import sys
import os
from glob import glob
from numba import jit
import warnings

os.environ['KERAS_BACKEND']='tensorflow'

import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import pandas as pd

from gensim.models import Word2Vec, KeyedVectors
from gensim.models.doc2vec import TaggedDocument

from konlpy.utils import pprint

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, roc_curve,  accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale, MinMaxScaler, LabelEncoder
from sklearn.manifold import TSNE

import keras.backend.tensorflow_backend as K
from keras.preprocessing import sequence
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.layers import Input, Flatten, Dense, Embedding, embeddings, merge, Dropout, Activation,  LSTM, Bidirectional, SimpleRNN, GRU
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import SpatialDropout1D
from keras.utils import np_utils
from tensorflow.python.client import device_lib
from keras.layers.merge import dot

import xgboost as xgb

import matplotlib.pyplot as plt

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import Database_Handler as dh
import Basic_Module as bm

In [3]:
from ckonlpy.tag import Twitter
from konlpy.tag import Mecab
ct = Twitter()
mecab = Mecab()

## Stopwords

In [4]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

## Load Data

In [5]:
#Naver
naverData = pickle.load(open('./data/pre_data/stastics/for_statistics_Naver_from_mongodb.pickled','rb'))
naverData = pd.DataFrame.from_dict(naverData, orient = 'index')
naverData.reset_index(inplace = True)
naverData.rename(columns = {'index' : 'id'}, inplace = True)
#Daum
daumData = pickle.load(open('./data/pre_data/stastics/for_statistics_daum_from_mongodb.pickled','rb'))
daumData = pd.DataFrame.from_dict(daumData, orient = 'index')
daumData.reset_index(inplace = True)
daumData.rename(columns = {'index' : 'id'}, inplace = True)

print ('Naver : {}'.format(naverData.shape))
print ('Daum : {}'.format(daumData.shape))

Naver : (15120, 11)
Daum : (9372, 11)


In [6]:
extNaverData = naverData.loc[:, ['id', 'title', 'date', 'press', 'rank']].copy()
extNaverData['site'] = pd.Series(['Naver']*extNaverData.shape[0])
extDaumData = daumData.loc[:, ['id', 'title', 'date', 'press', 'rank']].copy()
extDaumData['site'] = pd.Series(['daum']*extDaumData.shape[0])

In [7]:
if sys.platform =='darwin':
    loadModelPath = '/Volumes/disk1/model/'
    classifierPath = '/Volumes/disk1/data/pre_data/classifier/'
    #newsPath = '/Volumes/data/pre_data/news_sentiment/'
    newsPath = './data/pre_data/news_sentiment/'
elif sys.platform =='win32':
    loadModelPath = 'd:/model/'
    classifierPath = 'd:/data/pre_data/classifier/'
    newsPath = './data/pre_data/news_sentiment/'

## TaggedDocument

In [8]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags sentiment')

## Word2Vec Model

### Twitter

In [9]:
taggerName = 'ct'

#### Train data set으로부터 TF-IDF Vectorizer을 만듦

In [10]:
trainName = './data/pre_data/train_test_Data/pre_data_train_for_word2vec_sentiment_by_ct.pickled'
train = pickle.load(open(trainName, 'rb'))
tfidf = bm.Build_tfidf(train)
del train

100%|██████████| 442359/442359 [00:01<00:00, 408836.54it/s] 


(442359, 159113)
vocab size : 159113


#### News to tagged Document

In [11]:
if os.path.isfile('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_word2vec_sentiment_analysis.pickled'):
    taggedDaumData = pickle.load(open('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_word2vec_sentiment_analysis.pickled', 'rb'))
else:
    taggedDaumData = bm.MakeTaggedDataDAUM2(daumData, TaggedDocument, ct, stopwords, 'daum')
    pickle.dump(taggedDaumData, open('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_word2vec_sentiment_analysis.pickled', 'wb'))

    
if os.path.isfile('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_word2vec_sentiment_analysis.pickled'):
    taggedNaverData = pickle.load(open('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_word2vec_sentiment_analysis.pickled', 'rb'))
else:
    taggedNaverData = bm.MakeTaggedDataDAUM2(naverData, TaggedDocument, ct, stopwords, 'naver')
    pickle.dump(taggedNaverData, open('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_word2vec_sentiment_analysis.pickled', 'wb'))

#### Load Model

In [12]:
model1 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-0_min_count-2_by-ct.model')
model2 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-1_min_count-2_by-ct.model')
model3 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-1_cbow_mean-0_min_count-2_by-ct.model')

#### model1

In [13]:
modelName = bm.Return_ModelName('word2vec', model1,'ct')

In [14]:
classifierList = glob(classifierPath+'*'+modelName)

In [15]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

XGBoost
NeuralNetwork_2
NeuralNetwork_1
LogisticRegression
SVC
RandomForestClassifier


In [16]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedDaumData)

  7%|▋         | 10639/162640 [00:00<00:01, 106351.81it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 87107.52it/s]
1it [00:00,  6.78it/s]

running time : 0:00:01.873118
Vectorizing Data


9372it [01:38, 95.39it/s]


scaling Data
total running time : 0:01:40.614501


In [17]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 354020.47it/s]
9372it [00:00, 498345.78it/s]
9372it [00:00, 616331.66it/s]


CPU times: user 2.83 s, sys: 560 ms, total: 3.39 s
Wall time: 3.56 s


In [18]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedNaverData)

 10%|▉         | 15592/162640 [00:00<00:00, 155881.13it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 120517.20it/s]
7it [00:00, 67.10it/s]

running time : 0:00:01.353655
Vectorizing Data


15120it [02:38, 95.53it/s]


scaling Data
total running time : 0:02:40.678613


In [19]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extDaumData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 679639.88it/s]
15120it [00:00, 692909.80it/s]
15120it [00:00, 416587.03it/s]


CPU times: user 4.26 s, sys: 782 ms, total: 5.04 s
Wall time: 4.99 s


In [20]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model2

In [21]:
modelName = bm.Return_ModelName('word2vec', model2,'ct')

In [22]:
classifierList = glob(classifierPath+'*'+modelName)

In [23]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [24]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedDaumData)

  4%|▎         | 5706/162640 [00:00<00:02, 57040.06it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 91463.69it/s]
0it [00:00, ?it/s]

running time : 0:00:01.781403
Vectorizing Data


9372it [01:26, 108.67it/s]


scaling Data
total running time : 0:01:28.383311


In [25]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 780467.32it/s]
9372it [00:00, 727002.35it/s]
9372it [00:00, 787692.71it/s]


CPU times: user 26.1 s, sys: 1.39 s, total: 27.5 s
Wall time: 26 s


In [26]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedNaverData)

  9%|▉         | 15026/162640 [00:00<00:00, 150220.75it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 123188.23it/s]
8it [00:00, 76.91it/s]

running time : 0:00:01.324545
Vectorizing Data


15120it [03:19, 75.61it/s] 


scaling Data
total running time : 0:03:21.888815


In [27]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extDaumData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 526853.45it/s]
15120it [00:00, 713835.69it/s]
15120it [00:00, 612620.64it/s]


CPU times: user 43.5 s, sys: 1.53 s, total: 45 s
Wall time: 2min 59s


In [28]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model3

In [29]:
modelName = bm.Return_ModelName('word2vec', model3,'ct')

In [30]:
classifierList = glob(classifierPath+'*'+modelName)

In [31]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [32]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedDaumData)

  7%|▋         | 11311/162640 [00:00<00:01, 113080.45it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 123362.40it/s]
11it [00:00, 106.62it/s]

running time : 0:00:01.321223
Vectorizing Data


9372it [08:25, 18.53it/s]


scaling Data
total running time : 0:08:27.615345


In [33]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 545103.06it/s]
9372it [00:00, 551875.91it/s]
9372it [00:00, 759667.93it/s]


CPU times: user 31.5 s, sys: 1.33 s, total: 32.9 s
Wall time: 3min 54s


In [34]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedNaverData)

  8%|▊         | 13216/162640 [00:00<00:01, 132127.36it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 111925.33it/s]
8it [00:00, 75.92it/s]

running time : 0:00:01.458616
Vectorizing Data


15120it [12:43, 19.79it/s]


scaling Data
total running time : 0:12:46.090557


In [35]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extDaumData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 725347.72it/s]
15120it [00:00, 598050.53it/s]
15120it [00:00, 605109.31it/s]


CPU times: user 54.5 s, sys: 834 ms, total: 55.3 s
Wall time: 4min 19s


In [36]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

### Mecab

In [37]:
taggerName = 'mecab'

#### Train data set으로부터 TF-IDF Vectorizer을 만듦

In [38]:
trainName = './data/pre_data/train_test_Data/pre_data_train_for_word2vec_sentiment_by_mecab.pickled'
train = pickle.load(open(trainName, 'rb'))
tfidf = bm.Build_tfidf(train)
del train

100%|██████████| 442359/442359 [00:01<00:00, 254323.43it/s]


(442359, 161590)
vocab size : 161590


#### News to tagged Document

In [39]:
if os.path.isfile('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_word2vec_sentiment_analysis.pickled'):
    taggedDaumData = pickle.load(open('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_word2vec_sentiment_analysis.pickled', 'rb'))
else:
    taggedDaumData = bm.MakeTaggedDataDAUM2(daumData, TaggedDocument, mecab, stopwords, 'daum')
    pickle.dump(taggedDaumData, open('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_word2vec_sentiment_analysis.pickled', 'wb'))

    
if os.path.isfile('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_word2vec_sentiment_analysis.pickled'):
    taggedNaverData = pickle.load(open('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_word2vec_sentiment_analysis.pickled', 'rb'))
else:
    taggedNaverData = bm.MakeTaggedDataDAUM2(naverData, TaggedDocument, mecab, stopwords, 'naver')
    pickle.dump(taggedNaverData, open('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_word2vec_sentiment_analysis.pickled', 'wb'))

#### Load Model

In [40]:
model1 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-0_min_count-2_by-ct.model')
model2 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-1_min_count-2_by-ct.model')
model3 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-1_cbow_mean-0_min_count-2_by-ct.model')

#### model1

In [41]:
modelName = bm.Return_ModelName('word2vec', model1,'mecab')

In [42]:
classifierList = glob(classifierPath+'*'+modelName)

In [43]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [44]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedDaumData)

  6%|▌         | 9419/162640 [00:00<00:01, 94166.52it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 92502.60it/s]
1it [00:00,  7.35it/s]

running time : 0:00:01.770985
Vectorizing Data


9372it [08:40, 18.01it/s]


scaling Data
total running time : 0:08:42.419539


In [45]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 620711.16it/s]
9372it [00:00, 398979.10it/s]
9372it [00:00, 710433.88it/s]


CPU times: user 31.7 s, sys: 2.27 s, total: 34 s
Wall time: 4min 14s


In [46]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedNaverData)

  6%|▌         | 9654/162640 [00:00<00:01, 96501.44it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 134408.35it/s]
6it [00:00, 50.43it/s]

running time : 0:00:01.219531
Vectorizing Data


15120it [35:40,  7.06it/s]


scaling Data
total running time : 0:35:42.865940


In [47]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extDaumData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 397526.98it/s]
15120it [00:00, 671849.36it/s]
15120it [00:00, 739981.29it/s]


CPU times: user 43.3 s, sys: 2.22 s, total: 45.5 s
Wall time: 8min 53s


In [48]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model2

In [49]:
modelName = bm.Return_ModelName('word2vec', model2,'mecab')

In [50]:
classifierList = glob(classifierPath+'*'+modelName)

In [51]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [52]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedDaumData)

  8%|▊         | 12350/162640 [00:00<00:01, 123471.56it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 138500.01it/s]
12it [00:00, 117.01it/s]

running time : 0:00:01.177842
Vectorizing Data


9372it [18:11,  8.58it/s] 


scaling Data
total running time : 0:18:13.163216


In [53]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 838574.47it/s]
9372it [00:00, 792696.31it/s]
9372it [00:00, 550808.75it/s]


CPU times: user 29.5 s, sys: 753 ms, total: 30.2 s
Wall time: 29.1 s


In [54]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedNaverData)

  7%|▋         | 10887/162640 [00:00<00:01, 108834.04it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 118905.22it/s]
5it [00:00, 48.84it/s]

running time : 0:00:01.374156
Vectorizing Data


15120it [23:29, 10.72it/s]


scaling Data
total running time : 0:23:32.024969


In [55]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extDaumData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 510828.91it/s]
15120it [00:00, 706873.65it/s]
15120it [00:00, 544079.24it/s]


CPU times: user 44 s, sys: 1.73 s, total: 45.8 s
Wall time: 11min 47s


In [56]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model3

In [57]:
modelName = bm.Return_ModelName('word2vec', model3,'mecab')

In [58]:
classifierList = glob(classifierPath+'*'+modelName)

In [59]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [60]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedDaumData)

  7%|▋         | 10619/162640 [00:00<00:01, 106136.96it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 122429.06it/s]
8it [00:00, 68.54it/s]

running time : 0:00:01.334822
Vectorizing Data


9372it [09:26, 16.53it/s] 


scaling Data
total running time : 0:09:28.529569


In [61]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 597392.40it/s]
9372it [00:00, 649478.18it/s]
9372it [00:00, 709088.27it/s]


CPU times: user 31.3 s, sys: 945 ms, total: 32.2 s
Wall time: 9min 57s


In [62]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedNaverData)

  9%|▉         | 15110/162640 [00:00<00:00, 151062.69it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 148613.16it/s]
9it [00:00, 87.82it/s]

running time : 0:00:01.099475
Vectorizing Data


15120it [22:56, 10.99it/s]


scaling Data
total running time : 0:22:58.136819


In [63]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extDaumData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 559043.34it/s]
15120it [00:00, 610415.30it/s]
15120it [00:00, 600360.46it/s]


CPU times: user 58.1 s, sys: 762 ms, total: 58.8 s
Wall time: 56.9 s


In [64]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict